In [141]:
from PIL import Image
import numpy as np
from numpy import genfromtxt
from sklearn.decomposition import PCA, TruncatedSVD
import random
#ORDEN DE REVISIÓN SUGERIDO: 
# 1) kmeans()
# 2) load()
# 3) iteration()
# 4) distance()
#EL MAIN SE ENCUENTRA AL PIE DEL ARCHIVO


def distance(a, b):
    #DISTANCIA EUCLIDIANA N DIMENSIONAL
    aux = np.sum(np.square(a - b))
    return np.sqrt(aux)


def iteration(k_puntos, puntos, dim_datos):
    #CALCULO DE DISTANCIA DE CADA DATO CON LOS K CENTROIDES DE LOS CLUSTERES Y ASIGNAMIENTO DEL CLUSTER MAS CERCANO PARA CADA DATO
    clusters = []
    for idx, punto in enumerate(puntos):
        d = 99999999999991
        ki = 0
        for idx_k, k_i in enumerate(k_puntos):
            if(d > distance(punto, k_i)):
                d = distance(punto, k_i)
                ki = idx_k
        clusters.append(ki)




    #CONTABILIZACION DE ELEMENTOS POR CADA CLUSTER Y SUMA DE FEATURES PARA CADA UNO DE ELLOS
    aux = [0] * dim_datos
    means = [np.array(aux)] * len(puntos)
    count = [0] * len(puntos)
    for i in range(0,len(puntos)):
        idx = clusters[i]
        means[idx] = np.add(puntos[i], means[idx])
        count[idx] += 1


    #CÁLCULO DE LA MEDIA PARA CADA CLUSTER Y REASIGNACIÓN DE LOS K CENTROIDES
    k_p = []
    for j in range(len(means)):
        # print(means[i])
        # print(count[i])
        # break
        if(count[j] != 0):
            #print('a')
            for i in range(len(means[j])):
                means[j][i] = means[j][i]/count[j]
            k_p.append(np.array(means[j]))
    k_puntos = k_p

    #RETORNO DE DATOS
    return k_puntos, clusters

def load(k, redux,redux_type):

    #PROCESAMIENTO DE DATOS
    borrarcomillas = lambda x: x.replace('"', '')
    names = np.genfromtxt('clase.csv', dtype=None, delimiter=",", skip_header=1, usecols=1, encoding='utf-8', converters={1: borrarcomillas})
    data = np.genfromtxt('dataset.csv', delimiter=",")

    #REDUCCION DE DIMENSIONALIDAD
    svd = TruncatedSVD(n_components=redux)
    pca = PCA(n_components=redux)
    datos = data
    if redux_type == 0:
        datos = svd.fit(data).transform(data)
    elif redux_type == 1:
        datos = pca.fit(data).transform(data)


    '''
    maximo = 0
    minimo = 9999999999999
    for i in range(len(datos)):
        for j in range(len(datos[i])):
            #datos[i][j] = 1.08**datos[i][j]
            if maximo < datos[i][j]:
                maximo = datos[i][j]
            if minimo > datos[i][j]:
                minimo = datos[i][j]
    '''

    #VARIABLES UTILES
    n_datos = len(datos)
    dim_datos = len(datos[0])
    
    #CARGA DE PUNTOS
    puntos = []
    k_puntos = []
    for i in range(n_datos):
        puntos.append(np.array(datos[i]))

    #OBTENCIÓN DE PROMEDIOS DE CADA FEATURE Y SU DESVIACIÓN ESTANDAR

    data_ = datos.transpose()
    sd = []
    promedios = []
    for x in data_:
        sd.append(np.std(x))
        promedios.append(np.mean(x))
    

    #GENERACION DE LOS K PUNTOS PARA LOS CLUSTERES USANDO UN RANGO DETERMINADO POR EL PROMEDIO + Q*DESVIACION ESTANDAR DONDE Q ES UN NUMERO ARBITRARIO PERO QUE TENGA SENTIDO
    for i in range(k):
        aux = []
        for i in range(dim_datos):
            aux.append(random.randint(round(promedios[i]-2*sd[i]),round(promedios[i]+2*sd[i])))
        k_puntos.append(np.array(aux))

  
    #RETORNO DE LOS K PUNTOS PARA LOS CLUSTERES, LOS PUNTOS CARGADOS, LA DIMENSION DE LOS DATOS Y LOS LABELS CORRECTOS
    return k_puntos, puntos, dim_datos, names



def kmeans(k, epoch, redux, redux_type):

    #CARGA DE DATOS
    k_puntos, puntos, dim_datos, names = load(k, redux, redux_type)

    
    #MECANISMO PARA IMPRIMIR PORCENTAJE ACTUAL DE LA OPERACIÓN Y SABER CUANTO APROXIMADAMENTE FALTA PARA QUE TERMINE EL ALGORITMO
    suma = 0
    if(epoch >= 100):
        per = epoch / 100
        suma = 1
    else:
        per = 100/epoch
        suma = per
        
    tiempo = 0
    count = 0


    #LLAMADA AL ALGORITMO iteration() CON ITERACIONES DE REFINAMIENTO=EPOCH E IMPRESIÓN DE PORCENTAJE ACTUAL DE EJECUCIÓN DEL ALGORITMO
    clusters = []
    for i in range(epoch):
        if tiempo >= per:
            print(str(count)+'%')
            count += suma
            tiempo = 0
        k_puntos,clusters = iteration(k_puntos, puntos, dim_datos)
        tiempo+=suma
    print(str(100)+'%')  
            
    
    #FORMATEO CORRECTO DEL RESULTADO PARA DISPLAY 
    results = []
    for i in range(k):
        aux = []
        for j in range(len(clusters)):
            if clusters[j] == i:
                aux.append(names[j])
        results.append(aux)
    
    #IMPRESIÓN DE RESULTADOS
    count = 0
    for i in results:
        print('Cluster:', count, '\n')
        print(i)
        count +=1

    #RETORNO DE RESULTADOS
    return clusters




#MAIN
# k = cantidad de clusteres
# epochs = cantidad de iteraciones de 'refinación'
# redux = cantidad de dimensiones finales que tendrá el dataset (pca puede quejarse si le pones mas de N, siendo N la cantidad de datos)
# redux_type 0 = svd, 1 = pca
# kmeans() retorna un arreglo arr[] en donde cada posición i representa el elemento i del dataset[] ya reducido y procesado, y arr[i] contiene el número de cluster que se le ha asignado. El arreglo names[i] contiene el label real del elemento dataset[i]
# la reducción de dimensionalidad se hace en la función load(), dejaré un indicador para que sea evidente
k = 7
epochs = 100
redux = 10
redux_type = 0 
kmeans(k,epochs, redux, redux_type)



maximos y minimos 1139.5931448542399 -84.87695888065039
0%
1%
2%
3%
4%
5%
6%
7%
8%
9%
10%
11%
12%
13%
14%
15%
16%
17%
18%
19%
20%
21%
22%
23%
24%
25%
26%
27%
28%
29%
30%
31%
32%
33%
34%
35%
36%
37%
38%
39%
40%
41%
42%
43%
44%
45%
46%
47%
48%
49%
50%
51%
52%
53%
54%
55%
56%
57%
58%
59%
60%
61%
62%
63%
64%
65%
66%
67%
68%
69%
70%
71%
72%
73%
74%
75%
76%
77%
78%
79%
80%
81%
82%
83%
84%
85%
86%
87%
88%
89%
90%
91%
92%
93%
94%
95%
96%
97%
98%
100%
Cluster: 0 

['liver', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver', 'liver']
Cluster: 1 

['cerebellum', 'cerebellum', 'cerebellum', 'cerebellum', 'cerebellum', 'cerebellum', 'cerebellum', 'cerebellum', 'cerebellum', 'cerebellum', 'cerebellum', 'cerebellum', 'cerebellum', 'cerebellum', 'cerebellum', 'cerebellum', 'cerebellum', 'cerebellum', 'cerebellum', 'cerebellum', 'cerebellum', 'cerebellum', '